<a href="https://colab.research.google.com/github/paragrafff22/dw_matrix_car/blob/master/day3_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tables
!pip install eli5

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance 

In [3]:
cd 'drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [5]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

#Dummy Model

In [6]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [14]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)

mean_absolute_error(y,y_pred)

39465.934630440985

In [15]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [17]:
df['price_currency'].value_counts(normalize = True)*100

PLN    99.80844
EUR     0.19156
Name: price_currency, dtype: float64

In [18]:
df = df[df['price_currency'] != 'EUR']
df.shape

(106290, 155)

#Features

In [20]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat+ SUFFIX_CAT] = factorized_values


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [22]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [25]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
score = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')
np.mean(score)

-19566.588937368328

In [27]:
m = DecisionTreeRegressor(max_depth =5)
m.fit(X, y)

imp = PermutationImportance(m, random_state = 42).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2546 ± 0.0048,param_napęd__cat
0.2020 ± 0.0091,param_faktura-vat__cat
0.1937 ± 0.0063,param_stan__cat
0.1429 ± 0.0083,param_rok-produkcji__cat
0.0638 ± 0.0028,param_moc__cat
0.0421 ± 0.0025,param_skrzynia-biegów__cat
0.0421 ± 0.0018,feature_kamera-cofania__cat
0.0288 ± 0.0053,param_marka-pojazdu__cat
0.0201 ± 0.0032,param_pojemność-skokowa__cat
0.0162 ± 0.0002,feature_bluetooth__cat
